In [1]:
import IPython
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# для расчетов над tdb
from pycalphad import Database, equilibrium, variables as v 

# для MCMC расчетов
# import pymc as pm  # пакет для MCMC расчетов 
# import arviz as az # пакет для работы с типом данных arviz
# import pytensor
# import pytensor.tensor as pt
# import theano
# theano.config.exception_verbosity = 'high' # должно выдавать подробное описание ошибки, но не помогает

# import aesara

# пути к tdb
cc10_path = "tdbs/CoCr-01Oik_with_new_functions.tdb"

# print(f"Running on PyMC v{pm.__version__}")

In [2]:
# np.__version__

In [3]:
def from_xarray_to_pandas(xarray_data, phase_str):
    cr_tuple = xarray_data.X.sel(component = 'CR').data[0][0]
    phase_tuple = xarray_data.Phase.data[0][0]
    t_tuple = xarray_data.T.data
    
    df_res = pd.DataFrame()
    df_res['T'] = ''
    df_res['phase'] = ''
    df_res['cr_conc'] = ''
    
    df = pd.DataFrame()
    df['T'] = t_tuple
    df['phase_1'] = ''
    df['phase_2'] = ''
    df['phase_3'] = ''
    df['cr_conc_1'] = ''
    df['cr_conc_2'] = ''
    df['cr_conc_3'] = ''
    
    # print(t_tuple, len(t_tuple))
    for i in range(len(t_tuple)):
        df.iloc[i, df.columns.get_loc('phase_1')] = phase_tuple[i][0][0]
        df.iloc[i, df.columns.get_loc('phase_2')] = phase_tuple[i][0][1]
        df.iloc[i, df.columns.get_loc('phase_3')] = phase_tuple[i][0][2]
        df.iloc[i, df.columns.get_loc('cr_conc_1')] = cr_tuple[i][0][0]
        df.iloc[i, df.columns.get_loc('cr_conc_2')] = cr_tuple[i][0][1]
        df.iloc[i, df.columns.get_loc('cr_conc_3')] = cr_tuple[i][0][2]
    
    df_1 = df[(df['phase_1'] == phase_str)]
    df_1 = df_1[['phase_1', 'cr_conc_1', 'T']]
    df_1.rename(columns = {'phase_1':'phase', 'cr_conc_1':'cr_conc'}, inplace = True)
    
    df_2 = df[(df['phase_2'] == phase_str)]
    df_2 = df_2[['phase_2', 'cr_conc_2', 'T']]
    df_2.rename(columns = {'phase_2':'phase', 'cr_conc_2':'cr_conc'}, inplace = True)
    
    df_res = pd.concat([df_1, df_2])
    
    return df_res

In [9]:
df_sigma_fcc = pd.read_excel('emp_data/sigma_fcc_allibert.xls')
# df_sigma_bcc = pd.read_excel('emp_data/sigma_bcc_allibert.xls')
# df_sigma_hcp = pd.read_excel('emp_data/sigma_hcp_allibert.xls')

df_sigma_fcc['T'] = df_sigma_fcc['T'].round(2)
df_sigma_fcc['cr_conc'] = df_sigma_fcc['cr_conc'].round(6)
df_sigma_fcc_sigma_old = df_sigma_fcc[(df_sigma_fcc['phase'] == 'sigma_old')].reset_index()
df_sigma_fcc_sigma_old

,index,cr_conc,T,phase
0,6,0.532019,1321.35,sigma_old
1,7,0.542469,1226.97,sigma_old
2,8,0.547749,1471.91,sigma_old
3,9,0.554050,1370.79,sigma_old
4,10,0.556156,1269.66,sigma_old
5,11,0.559283,1422.47,sigma_old


In [13]:
db10 = Database(cc10_path)
phases10 = list(db10.phases.keys())
press = 101325
elements = ['CR', 'CO', 'VA']
el_cnt = 1
T = df_sigma_fcc_sigma_old['T'].to_numpy()
phase = 'SIGMA_OLD'
y_obs = df_sigma_fcc_sigma_old['cr_conc'].values
conditions = {v.X('CR'):0.5, v.P: 101325, v.T: T, v.N: el_cnt}

In [14]:
y_obs

array([0.532019, 0.542469, 0.547749, 0.55405 , 0.556156, 0.559283])

# Сейчас реализовано так

In [11]:
eq_test_now = equilibrium(db10
                                            , elements
                                            , phase # тут передается только сигма фаза
                                            , conditions
                                        )
eq_res_now = from_xarray_to_pandas(eq_test_now, phase)
eq_res_now

,phase,cr_conc,T
0,SIGMA_OLD,0.5,1321.35
1,SIGMA_OLD,0.5,1226.97
2,SIGMA_OLD,0.5,1471.91
3,SIGMA_OLD,0.5,1370.79
4,SIGMA_OLD,0.5,1269.66
5,SIGMA_OLD,0.5,1422.47


# А должно быть видимо вот так?

In [12]:
eq_test_new = equilibrium(db10
                                            , elements
                                            , phases10 # тут передаются все фазы
                                            , conditions
                                        )
eq_res_new = from_xarray_to_pandas(eq_test_new, phase)
eq_res_new

,phase,cr_conc,T
0,SIGMA_OLD,0.547649,1321.35
1,SIGMA_OLD,0.540192,1226.97
2,SIGMA_OLD,0.567237,1471.91
3,SIGMA_OLD,0.554281,1370.79
4,SIGMA_OLD,0.541562,1269.66
5,SIGMA_OLD,0.56099,1422.47
